In [ ]:
import threading
from datetime import datetime, timedelta, timezone
import time


In [ ]:

class Scheduler:
    """
    A class to represent a schedule for jobs with start and end dates.
    
    Attributes:
    -----------
    threading : boolean 
        Whether or not to use threading while running the jobs 
    
    Methods:
    --------
    job():
        Method to create and configure a job
    run_all():
        Runs all jobs with or without threading
    """

    def __init__(self, threading=False, scstartdate = None, sctimezone=None):
        self.threads = threading  # determines if threadig should happen of no 
        self.startdatesc = datetime.strptime(scstartdate, '%Y-%m-%d %H:%M')  if scstartdate else datetime.strptime(datetime.now().strftime('%Y-%m-%d %H:%M'), '%Y-%m-%d %H:%M') # startdate for scheduler class if not passed it takes current time 

        
        if sctimezone is not None:
            # if timezone is passed as Scheduler's attribute it turns passed startdate for passed timezone time 
            self.timezonesc = timezone(timedelta(hours=sctimezone))
            self.startdatesc = self.startdatesc.astimezone(self.timezonesc)
        else:
            # in any other cases it is taken current timezone
            self.timezonesc = datetime.now().astimezone().tzinfo
            self.startdatesc = self.startdatesc.astimezone(self.timezonesc)
            
        # print(f'{self.startdatesc}\n')
        
        # validating that passed startdate isn't more than current time
        if  self.startdatesc.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")
        
        self.jobs = []  #idk if i am gonna use it at all 

    def job(self, jstartdate=None, jtimezone=None):
        ###### This whole logics ######
        # jtimezone = jtimezone if jtimezone else self.timezonesc
        # jstartdate = jstartdate if jstartdate else self.startdatesc
        if jstartdate is not None and jtimezone is None:
            if isinstance(jstartdate, str):
                jtimezone = self.timezonesc
                jstartdate = datetime.strptime(jstartdate, '%Y-%m-%d %H:%M').astimezone(jtimezone)
                # print(f'stdate has value, timezone none {jstartdate}')
            else:
                jtimezone = self.timezonesc
                jstartdate = jstartdate.astimezone(jtimezone)
                # print(f'stdate has value, timezone none but isnance dattetime{jstartdate}')
                
            # print(f'\n\n')
    
        elif jstartdate is not None and jtimezone is not None: 
            if isinstance(jstartdate, str):
                jtimezone = timezone(timedelta(hours=jtimezone))
                jstartdate = datetime.strptime(jstartdate, '%Y-%m-%d %H:%M').astimezone(jtimezone)
                # print(f'stdate has value, timezone has value {jstartdate}')
            else:
                jtimezone = timezone(timedelta(hours=jtimezone))
                jstartdate = jstartdate.astimezone(jtimezone)
                # print(f'stdate has value, timezone has value instance is dadetime {jstartdate}')
            # print(f'\n\n')
        elif jstartdate is None and jtimezone is not None:
            jstartdate = self.startdatesc
            if isinstance(jstartdate, str):
                jtimezone = timezone(timedelta(hours=jtimezone))
                jstartdate = datetime.strptime(jstartdate, '%Y-%m-%d %H:').astimezone(jtimezone)
                # print(f'stdate is non, timezone has value {jstartdate}')
            else:
                jtimezone = timezone(timedelta(hours=jtimezone))
                jstartdate = jstartdate.astimezone(jtimezone)
                # print(f'stdate is none, timezone has value instance is datetime {jstartdate}')
            print(f'\n\n')
        elif jstartdate is None and jtimezone is None:
            jstartdate = self.startdatesc
            if isinstance(jstartdate, str):
                jtimezone = self.timezonesc
                jstartdate = datetime.strptime(jstartdate, '%Y-%m-%d %H:').astimezone(jtimezone)
                # print(f'stdate is non, timezone is none {jstartdate}')
            else:
                jtimezone = self.timezonesc
                jstartdate = jstartdate.astimezone(jtimezone)
                # print(f'stdate is non, timezone is none isntance is datetime {jstartdate}')
            # print(f'\n\n')
        else:
            raise AttributeError(f"You need to check Job's passed attributes, there is mismatch")

        # validating that passed startdate isn't more than current time
        if  jstartdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Job startdate cannot be less than the current date")

        job = {
            'startdate': jstartdate,
            'time_zone': jtimezone,
            'enddate': None,
            'func': None,
            'name': None,
            'unit': None,
            'interval': None,
            'next_run': None,
            'args': (),
            'kwargs': {}
        }
        class JobWrapper:
            def __init__(self, job):
                self.job = job
            @property
            def second(self):
                self.job['unit'] = 'second'
                # print(f'dis is stdate {jstartdate}')
                return self
            
            def do(self, func, name):
                self.job['func'] = func
                self.job['name'] = name
                return self
            
            def every(self, interval):
                self.job['interval'] = interval
                return self

            def until(self, enddate):
                enddate = datetime.strptime(enddate, '%Y-%m-%d %H:%M')
                enddate = enddate.astimezone(self.job['time_zone'])
                if enddate < self.job['startdate']:
                    raise ValueError('Startdate must be earlier than Enddate')
                self.job['enddate'] = enddate
                return self

            def calculate_next_run(self, current_time):
            # Truncate the current_time to remove milliseconds
                current_time = current_time.replace(microsecond=0)
            
                if self.job['unit'] == 'second':
                    self.job['next_run'] = current_time + timedelta(seconds=self.job['interval'])
                else:
                    raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
            
                return self.job['next_run']

            def run(self, *args, **kwargs):
                # Wait until jstartdate occurs
                now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                if now < self.job['startdate']:
                    wait_time = (self.job['startdate'] - now).total_seconds()
                    print(f'Waiting for {wait_time} seconds to')
                    time.sleep(wait_time)
                    
    
                while True:
                    now = datetime.now(self.job['time_zone']).replace(microsecond=0)
    
                    if self.job['next_run'] is None or now >= self.job['next_run']:
                        if self.job['func']:
                            self.job['func'](*self.job['args'], **self.job['kwargs'])
                        self.calculate_next_run(now)
    
                    # Check if the current time has passed the end time
                    if self.job['enddate'] and now >= self.job['enddate']:
                        break
    
                    time.sleep(1)

                    # time.sleep(1)
        job_wrapper = JobWrapper(job)
        self.jobs.append(job_wrapper)
        return job_wrapper
    
    def run_all(self):
        if self.threads:
            threads = []
            for job in self.jobs:
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                thread.start()
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()




In [ ]:


# Example functions
def report1(num1=1,num2 = 2):
    print(num1+num2)

# def report2():
#     print("Function report2 is running.")
# 
# def report3():
#     print("Function report3 is running.")

In [ ]:
scheduler = Scheduler(threading=True, sctimezone=5)

# scheduler.job(jstartdate='2024-09-24 22:12', jtimezone=4).do(report1, 'report1').every(1).second

scheduler.job(jtimezone=4).do(report1, 'report1').every(4).second.until('2024-08-23 21:23')
# scheduler.job(jstartdate='2024-08-22 17:10')
# scheduler.job()
# scheduler.job(jstartdate='2024-08-22 20:10', jtimezone=7)

scheduler.run_all()


In [ ]:
# ფუქნციას თუ არგუმენტები გადავეცი და ისე ვეცადე გაშვებას შტერდება
# დაამატე შემდეგი გაშვების თარიღის პრინტი